# Packages and necessary functions

In [ ]:
#The following packages are required to read and write files, as well as generate linear algebra calculations
using DelimitedFiles
using LinearAlgebra

#The following files with Julia functions are required
include("Operaciones_Basicas.jl")
include("Red_Dual.jl")
include("Separacion_Arreglos_X_Y.jl")
include("Generador_Vecindades_Pto_Arbitrario.jl")
include("voronoi.jl")
include("Algoritmo_Radio.jl")
include("Algoritmo_Voronoi.jl")
include("Algoritmo_Cluster_Principal_Iterado.jl")
include("Algoritmo_Parche_Cuadrado.jl")
include("Dentro_Fuera_Poligono.jl")
include("Diccionario_Indices_Centroides_Voronoi.jl")
include("Algoritmo_Dinamica_FFD.jl")

#### We load the functions that determine the trajectory of the particle when entering and leaving the Voronoi polygons

In [ ]:
include("Esferas_Duras_Trayectoria_Recta.jl")

In [ ]:
#We verify the size of the obstacle to consider
Radio_Obstaculo

#### We generate the parameters that determine the type of quasiperiodic arrangement that we are going to build

In [ ]:
NSides = 13; #Rotational symmetry of the quasiperiodic lattice
Error_Margin = 3; #Margin of error in the integers "n" associated with the star vectors of the GDM
Star_Vectors = [[BigFloat(1),0]]; #Arrangement that will contain the star vectors
for i in 1:(NSides-1)
    push!(Star_Vectors, [cos((2*i)*pi/NSides), sin((2*i)*pi/NSides)]); #Vertices of the polygon with "NSides" sides
end
Alphas_Array = fill(0.2, NSides); #Array with the alpha constants of the GDM
Average_Distance_Stripes = fill(NSides/2, NSides); #Array with the average spacing between stripes

#We load the information to generate the "circular" patches around an arbitrary point
Patch_Information = readdlm("Informacion_Parche_N_$(NSides).csv", BigFloat);
Reduction_Factor = 0.5; #Factor to generate a "safe radius"

#### We define conditions on the flight of a particle

In [ ]:
Long_Fly_Time = 5000000; #Maximum flight time that each particle will have
Number_Segments = 10; #Number of flights iterations to be calculated
Number_Free_Flies = 1000; #Number of free flights to be calculated for each iteration

#### We obtain the first collision of a particle and its velocity after the collision

In [ ]:
function first_Collition(Error_Margin, Average_Distance_Stripes, Star_Vectors, Alphas_Array, Patch_Information, Reduction_Factor, Long_Fly_Time)
    Array_Free_Fly = []; #Array where the length of the particle's first free flight will be saved
    Initial_Position_Obstacle = [0.0, 0.0]; #Position of a particle after colliding with an obstacle
    Initial_Velocity_Obstacle = [0.0, 0.0]; #Velocity of a particle after colliding with an obstacle
    
    APoint_Initial = [0.0,0.0]; #Array that will contain the initial position of the particles
    Counter_Iterations = 0; #Counter for the number of attempts made to find a valid starting position.
    SL = 1e6; #Size of the half side of the square centered on the (0,0) where a starting position is sought.
    
    #We look for a valid starting position in 100 attempts, if it is not found, it starts at (0,0)
    while Counter_Iterations < 100
        Counter_Iterations += 1;
        Test_Point = punto_Arbitrario(SL); #We generate an arbitrary point inside the square centered on the (0,0)
        Exclusion_Radius = 2; #Radius of a circle centered on Test_Point to filter possible container polygons
        
        #We generate the local neighborhood within the circle with radius "Exclusion_Radius" around the Test_Point point
        Dual_Points_R = region_Local_Radio(Error_Margin, Exclusion_Radius, Average_Distance_Stripes, Star_Vectors, Alphas_Array, Test_Point);
        X_R,Y_R = separacion_Arreglo_de_Arreglos_2D(Dual_Points_R);
        
        #We obtain the vertices of the container polygon from Test_Point
        Vertices_Container_Polygon = poligono_Contenedor_Radio(X_R,Y_R,Test_Point);
        
        #We check if Test_Point falls into some obstacle centered on the vertices of the container polygon
        if posicion_Valida_Obstaculos_Circulares(Test_Point, Vertices_Container_Polygon)
            APoint_Initial = Test_Point; break #We redefine the starting point and close the While loop
        end
    end
    
    println("The initial position is $(APoint_Initial)")
    
    #Algorithm that calculates the trajectories of the different particles
    Initial_Position_Obstacle, Initial_Velocity_Obstacle = first_Fly(Patch_Information, Reduction_Factor, Average_Distance_Stripes, Star_Vectors, Alphas_Array, APoint_Initial, Long_Fly_Time, Array_Free_Fly, Initial_Position_Obstacle, Initial_Velocity_Obstacle, cambio_Celda_Esferas_Duras_Trayectoria_Recta, estado_Tras_Tiempo_Rectas);

    return Initial_Position_Obstacle, Initial_Velocity_Obstacle
end

In [ ]:
Initial_Position_Obstacle, Initial_Velocity_Obstacle = first_Collition(Error_Margin, Average_Distance_Stripes, Star_Vectors, Alphas_Array, Patch_Information, Reduction_Factor, Long_Fly_Time);

#### Let us obtain the subsequent collisions of the particle

In [ ]:
function multiple_Collitions(Number_Segments, Number_Free_Flies, Average_Distance_Stripes, Star_Vectors, Alphas_Array, Patch_Information, Reduction_Factor, Long_Fly_Time, Initial_Position_Obstacle, Initial_Velocity_Obstacle)
    Free_Fly_Global = []; #Array where all free flight times will go

    for δ in 1:Number_Segments
        #We iterate on the number of collisions with obstacles we are looking for
        for γ in 1:Number_Free_Flies
            tiempo1 = time()
            open("Estado_FFD_N_13_Vuelos_Libres_1.txt", "w") do file
                write(file, "Estamos calculando el vuelo libre $(γ) de la iteración $(δ).")
            end

            Array_Free_Fly = []; #Arreglo where particle free flight length will be saved
            #We define the initial position as the last position of the particle when colliding with an obstacle
            APoint_Initial = copy(Initial_Position_Obstacle);

            Initial_Position_Obstacle, Initial_Velocity_Obstacle = multiple_Flies(Patch_Information, Reduction_Factor, Average_Distance_Stripes, Star_Vectors, Alphas_Array, APoint_Initial, Long_Fly_Time, Array_Free_Fly, Initial_Position_Obstacle, Initial_Velocity_Obstacle, cambio_Celda_Esferas_Duras_Trayectoria_Recta, estado_Tras_Tiempo_Rectas)

            #We save the length of the free flight that we just performed
            Free_Fly_Global = vcat(Free_Fly_Global, Array_Free_Fly)
            tiempo2 = time()
            println(tiempo2 - tiempo1)
        end

        writedlm("FFD_N_13_Vuelos_Libres_1.csv", Free_Fly_Global)
    end
    
    return Free_Fly_Global
end

In [ ]:
Array_Free_Flies = multiple_Collitions(Number_Segments, Number_Free_Flies, Average_Distance_Stripes, Star_Vectors, Alphas_Array, Patch_Information, Reduction_Factor, Long_Fly_Time, Initial_Position_Obstacle, Initial_Velocity_Obstacle);